#**Personalized Workout Recommender**



-Python Notebook: Application

* This project aims to create a personalized workout recommender system based on user input. Users describe their day and select a time of day (morning, afternoon, evening). Based on their mood and the time, the system suggests suitable workouts. The project uses Hugging Face’s sentiment analysis to classify the input text and a DataFrame to store possible workout suggestions


# 1. Setup and Dependencies

In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing i

In [2]:
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt
from transformers import pipeline

# 2. Text Classification Using Hugging Face

In [3]:
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# 3. Creating a Workout DataFrame


In [4]:
# Define workout recommendations based on mood and time of day
data = {
    'Mood': ['POSITIVE', 'NEGATIVE'],
    'Morning': ['Yoga', 'Stretching'],
    'Afternoon': ['HIIT', 'Strength Training'],
    'Evening': ['Meditation', 'Relaxing Walk']
}

# Convert to DataFrame
workout_df = pd.DataFrame(data)

# Gradio Interface

In [5]:
df = pd.DataFrame(columns=['Mood', 'Morning', 'Afternoon', 'Evening'])

def analyze_sentiment(text):
    result = classifier(text)[0]
    print(f"Classifier result: {result}")
    return result['label']

def recommend_workout(sentiment, time_of_day):
    time_of_day_mapping = {
        'morning': 'Morning',
        'afternoon': 'Afternoon',
        'evening': 'Evening'
    }
    time_of_day_formatted = time_of_day_mapping[time_of_day.lower()]
    workout = workout_df.loc[workout_df['Mood'] == sentiment, time_of_day_formatted].values
    return workout[0] if workout.size > 0 else "No workout found"

def append_user_input(df, sentiment, time_of_day, workout):
    time_of_day_mapping = {
        'morning': 'Morning',
        'afternoon': 'Afternoon',
        'evening': 'Evening'
    }
    time_of_day_formatted = time_of_day_mapping[time_of_day.lower()]
    new_row = pd.DataFrame([{'Mood': sentiment, 'Morning': None, 'Afternoon': None, 'Evening': None}])
    new_row[time_of_day_formatted] = workout
    df = pd.concat([df, new_row], ignore_index=True)
    return df

def process_input(mood_description, time_of_day):
    sentiment = analyze_sentiment(mood_description)
    recommended_workout = recommend_workout(sentiment, time_of_day)

    # Define color palette based on time of day and sentiment
    colors = {
        'morning': {'POSITIVE': 'yellow', 'NEGATIVE': 'darkorange'},
        'afternoon': {'POSITIVE': 'lightblue', 'NEGATIVE': 'darkblue'},
        'evening': {'POSITIVE': 'lightpink', 'NEGATIVE': 'red'}
    }

    time_color_map = colors[time_of_day.lower()]
    sentiment_color = time_color_map.get(sentiment, 'gray')

    # Append user input to the DataFrame
    global df
    df = append_user_input(df, sentiment, time_of_day, recommended_workout)

    # Return a styled string with the workout recommendation
    return f"<span style='color:{sentiment_color}; font-size:20px;'>Recommended workout: {recommended_workout}</span>"

# Gradio interface
def gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("""
        <h1 style='text-align: center;'>Personalized Workout Recommender System</h1>
        <p style='text-align: center;'>This system suggests personalized workouts based on your mood and the time of day. Enter your mood and time to get a workout recommendation, or explore visualizations of mood trends and workout preferences over time.</p>
        """)

        mood_description = gr.Textbox(label="Describe your day:", placeholder="e.g., I had a good day")
        time_of_day = gr.Dropdown(choices=["Morning", "Afternoon", "Evening"], label="Time of Day")
        output = gr.HTML(label="Recommended Workout")

        submit_button = gr.Button("Get Recommendation")
        submit_button.click(fn=process_input, inputs=[mood_description, time_of_day], outputs=output)

    return demo

# Launch the Gradio app
gradio_interface().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b716b2920a11995f6a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
import matplotlib.pyplot as plt
import io
import base64

def generate_time_graph():
    # Create a graph for time of day
    time_counts = df[['Mood', 'Morning', 'Afternoon', 'Evening']].count()

    fig, ax = plt.subplots()

    # Define colors for each time of day
    colors = {
        'Morning': 'yellow',
        'Afternoon': 'blue',
        'Evening': 'pink'
    }

    # Plot bars with respective colors
    bars = ax.bar(time_counts.index[1:], time_counts[1:], color=[colors.get(time, 'gray') for time in time_counts.index[1:]])
    ax.set_title('Workouts by Time of Day')
    ax.set_xlabel('Time of Day')
    ax.set_ylabel('Count')

    # Save plot to a BytesIO object and encode it as a base64 string
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    buf.close()

    return f"<img src='data:image/png;base64,{img_base64}'/>"

def generate_mood_graph():
    # Create a graph for mood distribution
    mood_counts = df['Mood'].value_counts()

    fig, ax = plt.subplots()

    # Define colors for each mood
    mood_colors = {
        'POSITIVE': 'lightgreen',
        'NEGATIVE': 'lightcoral'
    }

    # Plot bars with respective colors
    bars = ax.bar(mood_counts.index, mood_counts.values, color=[mood_colors.get(mood, 'gray') for mood in mood_counts.index])
    ax.set_title('Mood Distribution')
    ax.set_xlabel('Mood')
    ax.set_ylabel('Count')

    # Save plot to a BytesIO object and encode it as a base64 string
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    buf.close()

    return f"<img src='data:image/png;base64,{img_base64}'/>"

def gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("""
        <h1 style='text-align: center;'>Personalized Workout Recommender System</h1>
        <p style='text-align: center;'>This system suggests personalized workouts based on your mood and the time of day. Enter your mood and time to get a workout recommendation, or explore visualizations of mood trends and workout preferences over time.</p>
        """)

        with gr.Tab("Workout Recommendation"):
            mood_description = gr.Textbox(label="Describe your day:", placeholder="e.g., I had a long and stressful day")
            time_of_day = gr.Dropdown(choices=["Morning", "Afternoon", "Evening"], label="Time of Day")
            output = gr.HTML(label="Recommended Workout")

            submit_button = gr.Button("Get Recommendation")
            submit_button.click(fn=process_input, inputs=[mood_description, time_of_day], outputs=output)

        with gr.Tab("Graphs"):
            graph_type = gr.Radio(choices=["Time Graph", "Mood Graph"], label="Select Graph")
            graph_output = gr.HTML(label="Graph")

            def update_graph(selected_graph):
                if selected_graph == "Time Graph":
                    return generate_time_graph()
                elif selected_graph == "Mood Graph":
                    return generate_mood_graph()

            graph_type.change(fn=update_graph, inputs=graph_type, outputs=graph_output)

    return demo

# Launch the Gradio app
gradio_interface().launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c47e90760cde6be378.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
def print_dataframe(df):
    print("\nUpdated Workout DataFrame:")
    print(df)
print_dataframe(df)


Updated Workout DataFrame:
       Mood     Morning Afternoon        Evening
0  POSITIVE        None      HIIT           None
1  POSITIVE        None      None     Meditation
2  NEGATIVE        None      None  Relaxing Walk
3  NEGATIVE        None      None  Relaxing Walk
4  NEGATIVE  Stretching      None           None
5  POSITIVE        Yoga      None           None
6  POSITIVE        None      HIIT           None
7  POSITIVE        None      None     Meditation
8  POSITIVE        None      HIIT           None
9  POSITIVE        None      HIIT           None
